In [34]:

import pandas as pd
import numpy as np

from tqdm import tqdm
from textwrap import wrap


In [35]:
cd ~

/data/home/asurikov


In [36]:
!ls

ASPE-NEWS  ASTE  README.ipynb  cherniak  tmp


In [87]:
model_name = "ASTE/rugpt3large_based_on_gpt2_finetuned-50x3"
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

# Prepare data

In [48]:
data_path_pot = "ASTE/examples/pot.csv"
data_path_styler = "ASTE/examples/styler.csv"
data_path_buds = "ASTE/examples/buds.csv"
data_path_soundbar = "ASTE/examples/soundbar.csv"

res_path_pot = "ASTE/examples/evals/pot_eval"
res_path_styler = "ASTE/examples/evals/styler_eval"
res_path_buds = "ASTE/examples/evals/buds_eval"
res_path_soundbar = "ASTE/examples/evals/soundbar_eval"

In [49]:
data_pot = pd.read_csv(data_path_pot)
print(len(data_pot))
data_pot.head()

40


,text
0,Понравился дизайн- люблю чайники с прозрачным ...
1,"Красивый, с синей подсветкой, не громоздкий, ш..."
2,Очень эффектный чайничек. На подставке не заст...
3,"Плохой пластик. Как уже писали здесь, запах хи..."
4,"Думала, будет лучше. Зачеркнутая цена в 3000 э..."


In [50]:
data_styler = pd.read_csv(data_path_styler)
print(len(data_styler))
data_styler.head()

29


,text
0,"Оригинал, все проверила полностью. В составе 8..."
1,Ну я очень хотела такой стайлер и наконец муж ...
2,"Это популярный стайлер, оригинал однозначно и ..."
3,Только что получила. Как же я боялась подделки...
4,"У меня уже есть фен, покупала в фирменном мага..."


In [51]:
data_buds = pd.read_csv(data_path_buds)
data_soundbar = pd.read_csv(data_path_soundbar)

In [52]:
def format_input(text):
  return 'Текст: ' + " ".join(text.strip().split('\n')) + '\n'

texts_pot =    list(map(format_input, data_pot["text"]))
texts_styler = list(map(format_input, data_styler["text"]))
texts_buds =   list(map(format_input, data_buds["Текст"]))
texts_soundbar =   list(map(format_input, data_soundbar["text"]))

# Test run

In [88]:
prompt = "Текст: Позже были Huawei Freebuds Pro. Звук отличный, жесты есть. Но не сидели в ушах — напрягали, после чего уши болели. Плюс к этому у них был просто ужасный микрофон. Когда говорил на умеренно шумной улицы, собеседник слышал меня как из подушки или не слышал вообще.\n"

In [89]:

input_ids = tokenizer.encode(prompt, return_tensors="pt")
input_ids = input_ids.cuda()
out = model.generate(
        input_ids, 
        min_length=100, 
        max_length=300, 
        eos_token_id=5, 
        pad_token_id=1,
        top_k=1,
        top_p=0.0,
        no_repeat_ngram_size=5
)
generated_text = list(map(tokenizer.decode, out))[0]
#print('\n' + '\n'.join(wrap(prompt, 100)) + '\n')
print(generated_text[len(prompt):].split("\n\n")[0])
#print(generated_text[len(prompt):].split("\n"))

звук: отличный: положительно 
жесты: есть: положительно 
микрофон: ужасный: негативно 
микро: не сидели в ушах: негативно 
жизнь: не напрягали уши: положительно


# Evaluation

In [53]:
def run_dataset_eval_aste_joint(texts):
    results = []
    for index in tqdm(range(len(texts))):
        prompt = texts[index]

        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        input_ids = input_ids.cuda()
        out = model.generate(
                input_ids, 
                min_length=100, 
                max_length=350, 
                eos_token_id=5, 
                pad_token_id=1,
                top_k=2,
                top_p=0.0,
                no_repeat_ngram_size=5
        )
        generated_text = list(map(tokenizer.decode, out))[0]

        results.append(generated_text[len(prompt)-1:].strip().split("\n\n")[0])
    return results

In [88]:
pot_results = run_dataset_eval_aste_joint(texts_pot)
print(texts_pot[0])
print("Триплеты: \n" + pot_results[0])

100%|██████████| 40/40 [09:39<00:00, 14.49s/it]

Текст: Понравился дизайн- люблю чайники с прозрачным стеклом, красивая внутренняя подсветка кипящей воды , отлично,что без воды не работает- защита от включения без воды- безопасно. Хороший обьем- сразу на компанию 1,8 литров! Отличный чайник за небольшие деньги.Приехал сразу в новогодней упаковке- подарю на Новый Год!

Триплеты: 
дизайн: нравится: положительно 
подсветка: красивая: положительно 
объем: хороший: положительно 
чайник: отличный: положительно 
подарочная упаковка: в новой: положительно


In [89]:
styler_results = run_dataset_eval_aste_joint(texts_styler)
print(texts_styler[0])
print("Триплеты: \n" + styler_results[0])

100%|██████████| 29/29 [06:39<00:00, 13.79s/it]

Текст: Оригинал, все проверила полностью. В составе 8 насадок, подойдет для длинных и коротких волос. Волосы ниже талии, мне кажется, лучше взять лонг. Есть 3 скорости обдува и 3 режима подачи воздуха. Соответственно, самый горячий воздух это третий режим - очень хорошо закручивает локоны, укладка сохраняется надолго. Самый холодный воздух - он действительно дует прохладным, но с ним конечно локоны так уже не закручиваются. Есть щетка для чистки фильтра. ее все игнорируют, а зря. Надо чистить этой щеточкой стайлер, тогда он прослужит долгие годы, потому что не сказать, что у нас чистый воздух, у меня забивается пылью довольно быстро, раз в месяц чищу на всякий случай. У этой модели еще идет сумка, с ней я езжу в отпуск, очень удобно.

Триплеты: 
насадка: для длинных и коротких: положительно 
обдув: 3 режима: положительно 
режим подачи воздуха: 3 скорости: положительно 
щетка: идет в комплекте: положительно 
сумка: очень удобная: положительно 
стайлер: служит очень долго: положительно


In [46]:
buds_results = run_dataset_eval_aste_joint(texts_buds)
print(texts_buds[0])
print("Триплеты: \n" + buds_results[0])
with open(res_path_buds, "w") as file:
    file.write("\n\n".join(buds_results))

100%|██████████| 105/105 [20:46<00:00, 11.87s/it]

Текст: Маленький и удобный кейс и наушники. Удобное приложение со всеми необходимыми настройками и без излишеств. Звук более чем хороший! Приятные басы, звук не резкий. Режим "Звуковой фон" сделан будто для шпионов, которые чужие разговоры любят подслушивать. Что ж, буду пробовать :) Шумодав достойный! Качество сборки. Кейс не скрипит и открывается с небольшим усилием. Функция нахождения наушников — Самсунг, спасибо вам за это. Сидел в забегаловке и неосторожно вытащил наушники, не успев положить левый в кейс. Он выпал из рук и буквально укатился куда-то под большие диваны. Благо есть возможность "прозвонить". Прислушался — нашёл!

Триплеты: 
кейс: удобный, удобный: положительно 
входные устройства: удобные, удобные: положительно 
звук: более чем хороший: положительно 
функция нахождения наушников: Самсунг: положительно 
качество сборки: отличное: положительно 
шумодав: достойный: положительно


In [54]:
soundbar_results = run_dataset_eval_aste_joint(texts_soundbar)
print(texts_soundbar[0])
print("Триплеты: \n" + soundbar_results[0])
with open(res_path_soundbar, "w") as file:
    file.write("\n\n".join(soundbar_results))

100%|██████████| 108/108 [21:43<00:00, 12.07s/it]

Текст: Во-первых, надо трезво понимать, что за 3200 рублей купить многополосную акустику с супер детализацией и басами невозможно. Но сказать, что басов нет или детали совсем жёванные — будет неправдой. Во-вторых, брал этот саундбар для довольно старенького телевизора Haier, у которого родной звук, как из унитаза. После подключения саундбара разница была небо и земля. В третьих, и последнее. Самый главный аргумент — это бюджетность, поэтому буду честным, и оценку ставлю за соотношение цена-качество. + басы есть, + громкости хватает с лихвой, + присутствует голосовое меню, сообщающее о режиме подключения, + лампочка во время работы не горит (не отвлекает), + боковой джойстик регулировки стильный, как и сам саундбар.

Триплеты: 
басы: есть: положительно 
детализация: супер: положительно 
басы: есть: положительно


In [90]:
with open(res_path_pot, "w") as file:
    file.write("\n\n".join(pot_results))
with open(res_path_styler, "w") as file:
    file.write("\n\n".join(styler_results))

# Methods for analysis

In [7]:
def atomize_triplet(aspect, opinions, polarity):
  opinions = opinions.split(', ')
  return [[aspect, opinion, polarity] for opinion in opinions] 

def convert_polarity(span):
  if span == 'отрицательно' or span == 'негативно':
    return -1
  if span == 'нейтрально':
    return 0
  if span == 'положительно' or span == 'позитивно':
    return 1
  else:
    #error-code
    return -2

#loading in results
def load_ATE_results(path):
  results = ""
  with open(path, "r") as file:
    results = file.read()
  return [line.strip().split(', ') for line in results]

def load_ASTE_results(path):
  results = ""
  with open(path, "r") as file:
    results = file.read()
  
  #wrong_format
  wf = 0
  #right_format
  rf = 0
  #wrong_polarity - not in any class
  wp = 0
  #right_polarity
  rp = 0
  #wrong_polarity_format - polarity contains junk
  wpf = 0
  #right_polarity_format
  rpf = 0

  res_triplets = []

  for group in results.split('\n\n'):
    group_triplets = []
    for triplet in group.split('\n'):
      spans = [span.strip() for span in triplet.split(': ')]
      if len(spans) != 3:
        wf += 1
      else:
        rf += 1
        
        #we check polarity span - does it contain junk?
        polarity_span = spans[2].split()
        if (len(polarity_span) > 1):
          wpf += 1
          spans[2] = polarity_span[0]
        else:
          rpf += 1
        
        #we check polarity class
        polarity_converted = convert_polarity(spans[2])
        if polarity_converted == -2:
          wp += 1
        else:
          rp += 1

        group_triplets.extend(atomize_triplet(*spans))
    res_triplets.append(group_triplets)

  f_rate, p_rate, pf_rate = 0, 0, 0

  if wf + rf > 0:
    f_rate = wf / (wf + rf)
  if wp + rp > 0:
    p_rate = wp / (wp + rp)
  if wpf + rpf > 0:
    pf_rate = wpf / (wpf + rpf)
  
  return res_triplets, f_rate, p_rate, pf_rate

In [8]:
def print_triplets(tl):
  print("\n".join([": ".join(triplet) for triplet in tl]))

# Methods for similarity

In [32]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
st_model = SentenceTransformer('inkoziev/sbert_synonymy')

Defaulting to user installation because normal site-packages is not writeable


Some weights of the model checkpoint at /data/home/asurikov/.cache/torch/sentence_transformers/inkoziev_sbert_synonymy/ were not used when initializing BertModel: ['embeddings.position_ids']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#This method returns precision, recall, F1 from TP, FP, FN
def prf1(TP, FP, FN):
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    return precision, recall, F1

#This method finds longest common sequence
def LCS(a, b):
    if len(a) == 0 or len(b) == 0:
        return 0
    res = max(LCS(a[:-1], b), LCS(a, b[:-1]))
    if a[-1] == b[-1]:
        res = max(res, LCS(a[:-1], b[:-1]) + 1)
    return res

#This method takes two lists of strings and returnsut ids of left difference, intersection and right difference
#and a dictionary of ids in the right list that correspond to id from the left
#SBERT (st_model) and Transformer (tokenizer) tokenizers should be constructed before use
def get_span_ps_sets_ids(spans, p_ratio = 0.75, s_ratio = 0.5):
    #exit if one list is less than 2
    if len(spans)<2:
      return [], []

    indices = np.zeros(len(spans)).tolist()
    strings = np.full((len(spans)), "")

    for i in range(0, len(spans)-1):

        #Compute embeddings for both lists

        embs_curr = st_model.encode([spans[i]], convert_to_tensor=True)
        embs_right = st_model.encode(spans[i+1:], convert_to_tensor=True)
        tf_ids_left = tokenizer.encode(spans[i], return_tensors="pt").tolist()[0]
        tf_ids_right = [tokenizer.encode(span, return_tensors="pt").tolist()[0] for span in right]

        left_indices = set()
        right_indices = set()
        pairs = {}

        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embs_left, embs_right)

        for n in range(cosine_scores.size(0)):
          for m in range(cosine_scores.size(1)):

            #Compute longest common sequence
            lcs_len = LCS(tf_ids_left[n], tf_ids_right[m])
            lcs_ratio_left = lcs_len / len(tf_ids_left[n])
            lcs_ratio_right = lcs_len / len(tf_ids_right[m])

            #We find pairs of strings to be equal if LCS takes up more than [s_ratio] in one of tokenized
            #representations, or if SBERT is more than [r_ratio] confident in their similarity
            if (cosine_scores[n][m] >= p_ratio or lcs_ratio_left >= s_ratio or lcs_ratio_right >= s_ratio):
              left_indices.add(n)
              right_indices.add(m)
              if n not in pairs.keys():
                pairs[n] = [m]
              else:
                pairs[n].append(m)

    return 

#Basic check - works for all kids of tasks
def get_metrics_strict(pred, target):
  TP, FP, FN = 0, 0, 0
  for i in range(len(pred)):
    TP += len([x for x in pred[i] if x in target[i]])
    FP += len([x for x in pred[i] if x not in target[i]])
    FN += len([x for x in target[i] if x not in pred[i]])
  
  return TP, FP, FN

def convert_polarity(span):
  if span == 'отрицательно' or span == 'негативно':
    return -1
  if span == 'нейтрально':
    return 0
  if span == 'положительно' or span == 'позитивно':
    return 1
  else:
    #error-code
    return -2

#paraphrases & subsequenses
def get_metrics_ATE_ps(pred, target, p_ratio = 0.75, s_ratio = 0.5):
  TP, FP, FN = 0, 0, 0
  for i in range(len(pred)):
    left, intersection, right, pairs = get_span_ps_sets_ids(pred[i], target[i], p_ratio, s_ratio)
    TP += len(intersection)
    FP += len(left)
    FN += len(right)
  return TP, FP, FN


In [33]:
st_model.encode(["Hi darling", "LOL"], convert_to_tensor=True)

TypeError: forward() got an unexpected keyword argument 'return_dict'

# Load and analyze evals

In [9]:
res_triplets_pot, f_rate_pot, p_rate_pot, pf_rate_pot = load_ASTE_results(res_path_pot)
res_triplets_styler, f_rate_styler, p_rate_styler, pf_rate_styler = load_ASTE_results(res_path_styler)

model_name_short = model_name.split('/')[-1]
print(f"Model: {model_name_short}")

df_load = pd.DataFrame(columns=('format error', 'polarity error-class', 'polarity format error'))
df_load.loc[1] = [f_rate_pot, p_rate_pot, pf_rate_pot]
df_load.loc[2] = [f_rate_styler, p_rate_styler, pf_rate_styler]

df_load.index = ['SMM - pot reviews',
                'SMM - styler reviews']
df_load

Model: rugpt3large_based_on_gpt2_finetuned-50x3


,format error,polarity error-class,polarity format error
SMM - pot reviews,0.187879,0.0,0.000000
SMM - styler reviews,0.162963,0.0,0.026549


In [21]:
idx = 19

print(texts_styler[idx])
print("Триплеты:")
print_triplets(res_triplets_styler[idx])

Текст: В качестве подарка лучше не придумать, именинница рада и очень благодарна

Триплеты:
подарок: не придумать: положительно
подарок: именинка рада и очень благодарна: положительно


In [24]:
all_triplets_styler = []
for triplets in res_triplets_styler:
    all_triplets_styler.extend(triplets)

In [25]:
all_triplets_styler[0]

['насадка', 'для длинных и коротких', 'положительно']